# Imports

In [9]:
import pandas as pd
import numpy as np

raw_train = pd.read_csv("train.csv")
raw_test = pd.read_csv("test.csv")

raw_train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


# Exploratory Data Analysis

In [2]:
print('Pclass')
print('Range: ' + str(raw_test['Pclass'].min()) + '-' + str(raw_test['Pclass'].max()))
print('Missing Values: ' + str(raw_test['Pclass'].size - raw_test['Pclass'].count()))
print()
print('Sex')
print('Missing Values: ' + str(raw_test['Sex'].size - raw_test['Sex'].count()))
print()
print('Age')
print('Range: ' + str(raw_test['Age'].min()) + '-' + str(raw_test['Age'].max()))
print('Missing Values: ' + str(raw_test['Age'].size - raw_test['Age'].count()))
print()
print('SibSp')
print('Range: ' + str(raw_test['SibSp'].min()) + '-' + str(raw_test['SibSp'].max()))
print('Missing Values: ' + str(raw_test['SibSp'].size - raw_test['SibSp'].count()))
print()
print('Parch')
print('Range: ' + str(raw_test['Parch'].min()) + '-' + str(raw_test['Parch'].max()))
print('Missing Values: ' + str(raw_test['Parch'].size - raw_test['Parch'].count()))
print()
print('Fare')
print('Range: ' + str(raw_test['Fare'].min()) + '-' + str(raw_test['Fare'].max()))
print('Missing Values: ' + str(raw_test['Fare'].size - raw_test['Fare'].count()))
print()
print('Embarked')
print('Missing Values: ' + str(raw_test['Embarked'].size - raw_test['Embarked'].count()))

Pclass
Range: 1-3
Missing Values: 0

Sex
Missing Values: 0

Age
Range: 0.17-76.0
Missing Values: 86

SibSp
Range: 0-8
Missing Values: 0

Parch
Range: 0-9
Missing Values: 0

Fare
Range: 0.0-512.3292
Missing Values: 1

Embarked
Missing Values: 0


In [3]:
age = raw_train['Age'].floordiv(10)
age.index = age
print(age.mean())
age.groupby(level=0).count()

2.5252100840336134


,Age
Age,
0.0,62
1.0,102
2.0,220
3.0,167
4.0,89
5.0,48
6.0,19
7.0,6
8.0,1


In [4]:
fare = raw_train['Fare'].floordiv(50)
fare.index = fare
print(fare.mean())
fare.groupby(level=0).count()

0.3221099887766554


,Fare
Fare,
0.0,730
1.0,108
2.0,24
3.0,9
4.0,11
5.0,6
10.0,3


# Preprocessing

In [5]:
def transform(data,age_bin = 10,fare_bin = 50):
  sex = data['Sex'].map({'male':0,'female':1,np.nan:1})
  age = data['Age'].floordiv(age_bin,fill_value=data['Age'].mean())
  fare = data['Fare'].floordiv(fare_bin,fill_value=0)
  embarked = data['Embarked'].map({'S':0,'C':1,'Q':2})
  cleaned = pd.DataFrame({'Pclass':data['Pclass'],'Sex':sex,'Age':age,'SibSp':data['SibSp'],'Parch':data['Parch'],'Fare':fare,'Embarked':embarked})
  return cleaned

transform(raw_train)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,0,2.0,1,0,0.0,0.0
1,1,1,3.0,1,0,1.0,1.0
2,3,1,2.0,0,0,0.0,0.0
3,1,1,3.0,1,0,1.0,0.0
4,3,0,3.0,0,0,0.0,0.0
...,...,...,...,...,...,...,...
886,2,0,2.0,0,0,0.0,0.0
887,1,1,1.0,0,0,0.0,0.0
888,3,1,2.0,1,2,0.0,0.0
889,1,0,2.0,0,0,0.0,1.0


# Training

In [6]:
def entropy(data):
  p1 = data['Survived'].mean();
  p0 = 1 - p1
  if p0 == 0 or p1 == 0: return 0
  return -1 * (p0*np.log2(p0) + p1*np.log2(p1))

entropy(raw_train)

np.float64(0.9607079018756469)

In [20]:
def train(data,labels,max_depth,min_improv=0,random=False):
  if(max_depth > 0):
    H = entropy(data)
    if(H > 0):
      if not random:
        min = 2
        for l in labels:
          groupby = data.groupby(by=l)
          e = 0
          for value, group in groupby:
            e += entropy(group) * len(group)/len(data)
          if e < min:
            min = e
            label = l
            g = groupby
        if min < H - min_improv:
          labels.remove(label)
          next = dict();
          for value, group in g:
            next.update({value:train(group,labels,max_depth-1)})
          return {'label':label,'entropy':H,'next':next}
      else:
        i = np.random.randint(len(labels))
        l = labels[i]
        groupby = data.groupby(by=l)
        labels.remove(l)
        next = dict();
        for value, group in groupby:
          next.update({value:train(group,labels,max_depth-1)})
        return {'label':l,'entropy':H,'next':next}
  return {'label':'End','entropy':entropy(data),'class': 0 if data['Survived'].mean() < .5 else 1}

In [13]:
data = transform(raw_train)
data['Survived'] = raw_train['Survived']
labels = ['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']
model = train(data,labels,len(labels))
model

{'label': 'Sex',
 'entropy': np.float64(0.9607079018756469),
 'next': {0: {'label': 'Age',
   'entropy': np.float64(0.6991817891208407),
   'next': {0.0: {'label': 'SibSp',
     'entropy': np.float64(0.9744894033980523),
     'next': {0: {'label': 'End', 'entropy': 0, 'class': 1},
      1: {'label': 'End', 'entropy': 0, 'class': 1},
      2: {'label': 'End', 'entropy': 0, 'class': 1},
      3: {'label': 'End', 'entropy': 0, 'class': 0},
      4: {'label': 'Parch',
       'entropy': np.float64(0.4689955935892812),
       'next': {1: {'label': 'End', 'entropy': 0, 'class': 0},
        2: {'label': 'End',
         'entropy': np.float64(0.9182958340544896),
         'class': 0}}},
      5: {'label': 'End', 'entropy': 0, 'class': 0}}},
    1.0: {'label': 'Fare',
     'entropy': np.float64(0.5373760853377334),
     'next': {0.0: {'label': 'Embarked',
       'entropy': np.float64(0.46274905857817394),
       'next': {0.0: {'label': 'Pclass',
         'entropy': np.float64(0.4262286569981447),

In [78]:
def trainForest(data,max_depth, n_trees, frac=.5, random=True):
  models = []
  for i in range(n_trees):
    d = data.sample(frac=frac,replace=True)
    labels = ['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']
    models += [train(d,labels,max_depth,random=random)]
  return models

In [79]:
data = transform(raw_train)
data['Survived'] = raw_train['Survived']
labels = ['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']
models = trainForest(data,len(labels),10)
models

[{'label': 'Sex',
  'entropy': np.float64(0.9789513902784714),
  'next': {0: {'label': 'Pclass',
    'entropy': np.float64(0.7288903745523254),
    'next': {1: {'label': 'Age',
      'entropy': np.float64(0.9747785474909672),
      'next': {0.0: {'label': 'End', 'entropy': 0, 'class': 1},
       1.0: {'label': 'SibSp',
        'entropy': np.float64(0.9182958340544896),
        'next': {0: {'label': 'End', 'entropy': 0, 'class': 1},
         1: {'label': 'End', 'entropy': 0, 'class': 0}}},
       2.0: {'label': 'Fare',
        'entropy': np.float64(0.9798687566511528),
        'next': {0.0: {'label': 'Embarked',
          'entropy': np.float64(0.9967916319816366),
          'next': {0.0: {'label': 'End',
            'entropy': np.float64(0.9456603046006401),
            'class': 1},
           1.0: {'label': 'End',
            'entropy': np.float64(0.8112781244591328),
            'class': 0}}},
         1.0: {'label': 'Parch',
          'entropy': np.float64(0.8112781244591328),
      

# Testing

In [68]:
def testForest(data,models):
  correct = 0
  for i in data.index:
    vote = 0
    for model in models:
      layer = model
      while not layer['label'] == 'End':
        value = data[layer['label']][i]
        if value in layer['next']:
          layer = layer['next'][value]
        else:
          keys = list(layer['next'].keys())
          before = keys[0]
          for k in keys:
            after = k
            if(after > value): break
            before = k
          if after - value < value - before:
            layer = layer['next'][after]
          else:
            layer = layer['next'][before]
      vote += layer['class']
    vote = (0 if vote < 5 else 1)
    if vote == data['Survived'][i]: correct += 1
  return correct / len(data)

In [80]:
testForest(data,models)

0.8092031425364759

# Evaluation

In [81]:
data = transform(raw_train)
data['Survived'] = raw_train['Survived']
training = data.sample(frac=.8)
validation = data.drop(training.index)
for i in range(1,11):
  labels = ['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']
  models = trainForest(training,len(labels),10,i*.1)
  print(str(i) + ': ' + str(testForest(validation,models)))

1: 0.7865168539325843
2: 0.7921348314606742
3: 0.7584269662921348
4: 0.7640449438202247
5: 0.8033707865168539
6: 0.7696629213483146
7: 0.7752808988764045
8: 0.797752808988764
9: 0.7808988764044944
10: 0.797752808988764


In [82]:
data = transform(raw_train)
data['Survived'] = raw_train['Survived']
training = data.sample(frac=.8)
validation = data.drop(training.index)
for i in range(1,30):
  labels = ['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']
  models = trainForest(training,len(labels),i,.8)
  print(str(i) + ': ' + str(testForest(validation,models)))

1: 0.6179775280898876
2: 0.6179775280898876
3: 0.6179775280898876
4: 0.6179775280898876
5: 0.6910112359550562
6: 0.6629213483146067
7: 0.7640449438202247
8: 0.7865168539325843
9: 0.7865168539325843
10: 0.7921348314606742
11: 0.7808988764044944
12: 0.7752808988764045
13: 0.7752808988764045
14: 0.7640449438202247
15: 0.7471910112359551
16: 0.7584269662921348
17: 0.7359550561797753
18: 0.7359550561797753
19: 0.7303370786516854
20: 0.7303370786516854
21: 0.7528089887640449
22: 0.702247191011236
23: 0.7303370786516854
24: 0.7415730337078652
25: 0.702247191011236
26: 0.6853932584269663
27: 0.6741573033707865
28: 0.6460674157303371
29: 0.7191011235955056


In [83]:
data = transform(raw_train)
data['Survived'] = raw_train['Survived']
training = data.sample(frac=.8)
validation = data.drop(training.index)
for i in range(1,11):
  labels = ['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']
  models = trainForest(training,len(labels),10,i*.1,random=False)
  print(str(i) + ': ' + str(testForest(validation,models)))

1: 0.7696629213483146
2: 0.7696629213483146
3: 0.797752808988764
4: 0.797752808988764
5: 0.7808988764044944
6: 0.797752808988764
7: 0.797752808988764
8: 0.797752808988764
9: 0.797752808988764
10: 0.797752808988764


In [86]:
data = transform(raw_train)
data['Survived'] = raw_train['Survived']
training = data.sample(frac=.8)
validation = data.drop(training.index)
for i in range(1,30):
  labels = ['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']
  models = trainForest(training,len(labels),i,.8,random=False)
  print(str(i) + ': ' + str(testForest(validation,models)))

1: 0.6460674157303371
2: 0.6460674157303371
3: 0.6460674157303371
4: 0.6460674157303371
5: 0.7921348314606742
6: 0.7921348314606742
7: 0.8033707865168539
8: 0.797752808988764
9: 0.8033707865168539
10: 0.8033707865168539
11: 0.797752808988764
12: 0.797752808988764
13: 0.797752808988764
14: 0.797752808988764
15: 0.7921348314606742
16: 0.797752808988764
17: 0.7865168539325843
18: 0.7921348314606742
19: 0.7696629213483146
20: 0.7865168539325843
21: 0.7640449438202247
22: 0.797752808988764
23: 0.7865168539325843
24: 0.7921348314606742
25: 0.7865168539325843
26: 0.7471910112359551
27: 0.7752808988764045
28: 0.7640449438202247
29: 0.7808988764044944


In [87]:
data = transform(raw_train)
data['Survived'] = raw_train['Survived']
training = data.sample(frac=.8)
validation = data.drop(training.index)
labels = ['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']
for i in range(len(labels)):
  labels = ['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']
  models = trainForest(training,i,10,.8,random=False)
  print(str(i) + ': ' + str(testForest(validation,models)))

0: 0.6404494382022472
1: 0.8202247191011236
2: 0.8089887640449438
3: 0.8146067415730337
4: 0.8202247191011236
5: 0.8089887640449438
6: 0.8202247191011236


In [89]:
for a in range(1,100,1):
  data = transform(raw_train,age_bin=a)
  data['Survived'] = raw_train['Survived']
  training = data.sample(frac=.8)
  validation = data.drop(training.index)
  labels = ['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']
  models = trainForest(training,len(labels),10,.8,random=False)
  print(str(a) + ': ' + str(testForest(validation,models)))

1: 0.7134831460674157
2: 0.7921348314606742
3: 0.7865168539325843
4: 0.7584269662921348
5: 0.797752808988764
6: 0.7584269662921348
7: 0.7584269662921348
8: 0.7415730337078652
9: 0.797752808988764
10: 0.8314606741573034
11: 0.8202247191011236
12: 0.7752808988764045
13: 0.7865168539325843
14: 0.7921348314606742
15: 0.7752808988764045
16: 0.8314606741573034
17: 0.8089887640449438
18: 0.7471910112359551
19: 0.7752808988764045
20: 0.7134831460674157
21: 0.7471910112359551
22: 0.7528089887640449
23: 0.7528089887640449
24: 0.8258426966292135
25: 0.7865168539325843
26: 0.7584269662921348
27: 0.7303370786516854
28: 0.7808988764044944
29: 0.7865168539325843
30: 0.7640449438202247
31: 0.7696629213483146
32: 0.7696629213483146
33: 0.7808988764044944
34: 0.7640449438202247
35: 0.7696629213483146
36: 0.7808988764044944
37: 0.7471910112359551
38: 0.7471910112359551
39: 0.7640449438202247
40: 0.7865168539325843
41: 0.7303370786516854
42: 0.8202247191011236
43: 0.7808988764044944
44: 0.7696629213483146

In [91]:
for f in range(5,500,5):
  data = transform(raw_train,age_bin=10,fare_bin=f)
  data['Survived'] = raw_train['Survived']
  training = data.sample(frac=.8)
  validation = data.drop(training.index)
  labels = ['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']
  models = trainForest(training,len(labels),10,.8,random=False)
  print(str(f) + ': ' + str(testForest(validation,models)))

5: 0.8033707865168539
10: 0.797752808988764
15: 0.7752808988764045
20: 0.7921348314606742
25: 0.8202247191011236
30: 0.7696629213483146
35: 0.8258426966292135
40: 0.7359550561797753
45: 0.8033707865168539
50: 0.8202247191011236
55: 0.8089887640449438
60: 0.7921348314606742
65: 0.8146067415730337
70: 0.7808988764044944
75: 0.8314606741573034
80: 0.7921348314606742
85: 0.7752808988764045
90: 0.8258426966292135
95: 0.8314606741573034
100: 0.7808988764044944
105: 0.7696629213483146
110: 0.848314606741573
115: 0.797752808988764
120: 0.797752808988764
125: 0.8314606741573034
130: 0.8146067415730337
135: 0.8033707865168539
140: 0.7865168539325843
145: 0.8202247191011236
150: 0.8426966292134831
155: 0.8089887640449438
160: 0.7921348314606742
165: 0.7584269662921348
170: 0.7808988764044944
175: 0.797752808988764
180: 0.8539325842696629
185: 0.7921348314606742
190: 0.8764044943820225
195: 0.7696629213483146
200: 0.797752808988764
205: 0.8089887640449438
210: 0.7415730337078652
215: 0.80898876404

# Prediction

In [95]:
def predictForest(data,models):
  out = pd.DataFrame(columns=['PassengerId','Survived'])
  out['PassengerId'] = data['PassengerId']
  for i in data.index:
    vote = 0
    for model in models:
      layer = model
      while not layer['label'] == 'End':
        value = data[layer['label']][i]
        if value in layer['next']:
          layer = layer['next'][value]
        else:
          keys = list(layer['next'].keys())
          before = keys[0]
          for k in keys:
            after = k
            if(after > value): break
            before = k
          if after - value < value - before:
            layer = layer['next'][after]
          else:
            layer = layer['next'][before]
      vote += layer['class']
    vote = 0 if vote < 5 else 1
    out.loc[i,'Survived'] = vote
  return out

In [109]:
data = transform(raw_train,age_bin=10,fare_bin=100)
data['Survived'] = raw_train['Survived']
labels = ['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']
models = trainForest(data,len(labels),10,.8,random=False)

In [110]:
data = transform(raw_test,age_bin=5,fare_bin=20)
data['PassengerId'] = raw_test['PassengerId']
out = predictForest(data,models)
out.to_csv('RandomForest.csv', index = False)
out

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
